In [3]:
import os
import xgboost as xgb

In [61]:
import datetime
from sklearn import svm
import sklearn
from sklearn.preprocessing import Imputer
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score

In [5]:
Enrich_Path =r"C:\Users\Persona.ly_DS1\Desktop\Technion\Data Processing\2. Data Exploration and Cutoffs"
os.chdir(Enrich_Path)

### let's change a bit to OOP for better handling of the predictions  

In [6]:
os.getcwd()

'C:\\Users\\Persona.ly_DS1\\Desktop\\Technion\\Data Processing\\2. Data Exploration and Cutoffs'

In [7]:
%run ./Exploration.ipynb

Reading&Enrichments.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "# Reading the data and adding Enrichments"


In [8]:
df.head()

,time,bidtime,bidid,type,crid,deviceid,devlang,sspid,bundle,siteid,sitename,bid,impr,install,installprice,make,model,osv,isp,state,useroffset,height,uadevicetype,uabatterycapacity,uadeviceram,uareleaseyear,ppi,displaymanagerver,ssps,bundletotalfc,month,Impression_Price,bundleCategory,bundleAvgUserRating,is_landscape,rewarded,date,time_diff,local time,Local_day_of_week,normal_ssps,normal_rate,isLandscape,log2ofstorage,device_age,d_age_normalized,cat_is_casino
4,1.548845e+09,1548845084493,674a6c69-532e-45ac-8f07-81d60a9a06eb,impression,1468,df23d29a-3750-4789-8b2c-4cca6a36772e,en,chartboost,com.mindstormstudios.coinparty.google,561cb22af6cd4540a550863b,Zombie Ghosts Coin Party Dozer,NaN,1.0,0.0,0.0,LG,LM-V405,8.1,AT&T U-verse,IL,-06:00,480.0,SmartPhone,3300.0,6144.0,2018.0,536.92,6.6.1,4.0,2.0,1,0.023375,"games,games/casino",4.4,1,1,2019-01-30 10:45:28.733999968,06:00:00,2019-01-30 04:45:28.733999968,2.0,0.176471,0.850,0,6.0,1.0,0.111111,1
6,1.548845e+09,1548844905459,232a81e4-60cb-4e9b-a862-afa2a4a797a3,impression,1169,0bd1dc48-3b67-43c5-bc1f-106172d74790,en,unity_us,com.vegasslotspocket.heartofdiamond,df5e19e291e3e1b140cb9b70dcbd9ebc,Slots! CashHit Slot Machines & Casino Games Party,NaN,1.0,0.0,0.0,Samsung,SM-S327VL,6,West Central Telephone Assn,MN,-06:00,720.0,SmartPhone,2600.0,1536.0,2017.0,293.72,NaN,6.0,3.0,1,0.020130,"games,games/board,games/casino",4.7,1,0,2019-01-30 10:45:35.161000013,06:00:00,2019-01-30 04:45:35.161000013,2.0,0.294118,0.925,1,4.0,2.0,0.222222,1
11,1.548845e+09,1548845209785,6d9feae9-1d70-4429-97ec-f6ae6aad14a1,impression,1194,959e3d81-edde-4b01-a168-1b7a6f336eed,en,chartboost,com.zealgames.cashmania,5b39e39c6a0af32a0caed759,Cashmania Slots 2019: Free Vegas Casino Slot G...,NaN,1.0,0.0,0.0,Samsung,SM-J737P,8,Spectrum,WI,-06:00,320.0,SmartPhone,3300.0,2048.0,2018.0,267.02,7.3.1,10.0,3.0,1,0.020700,"games,games/casino",4.7,1,1,2019-01-30 10:46:53.872999907,06:00:00,2019-01-30 04:46:53.872999907,2.0,0.529412,0.925,1,5.0,1.0,0.111111,1
13,1.548845e+09,1548845211500,8f893eea-c785-48ef-98cb-67adf6c7395f,impression,1193,9852f122-18cb-4f26-b445-fbdf961eeff2,en,adcolony,com.gsn.android.casino,0343291e164bbc22dc7139eb918db7a9,GSN Casino - Mediation (no blocks),NaN,1.0,0.0,0.0,NaN,NaN,8.1,Spectrum,WI,-06:00,320.0,NaN,NaN,NaN,NaN,NaN,3.3.5,11.0,1.0,1,0.008312,"games,games/casino",4.3,1,1,2019-01-30 10:47:27.193000078,06:00:00,2019-01-30 04:47:27.193000078,2.0,0.588235,0.825,1,NaN,NaN,NaN,1
21,1.548845e+09,1548845308448,b5f53834-9373-4a15-9cbd-893522584329,impression,1469,e84ab04d-179c-4c5c-853a-b41a1d9f2dab,en,adcolony,com.fortmasongames.confetti,14a247c9b52706c521f7d51578244e4b,Lucky CASH Slots - Win Real Money & Prizes,NaN,1.0,0.0,0.0,Samsung,SM-G930A,8,Suddenlink Communications,AR,-06:00,480.0,SmartPhone,3000.0,4096.0,2016.0,575.92,3.2.0,11.0,2.0,1,0.041311,"games,games/casino",4.6,1,1,2019-01-30 10:48:52.743000031,06:00:00,2019-01-30 04:48:52.743000031,2.0,0.588235,0.900,0,5.0,3.0,0.333333,1


In [9]:
class dataset():
    #class of data set for prediction using train_test_ split and storage of model training

    def __init__(self,df, y,clf= None,
                 x_train=None, y_train=None, 
                 x_dev = None, y_dev=None,
                 y_dev_pred = None, y_pred = None,clf_log= None,clf_lgbm = None,clf_RF = None,clf_DT = None,
                 x_test=None, y_test=None, t_sep = 0.75,d_sep=0.13, ts_orient = 0 ):
        if ts_orient!=1:
            self.df = df
            self.y = y
            self.ratio = ratio
            
            test_sep = 1-(t_sep+d_sep)
            self.x_train , self.x_test , self.y_train, self.y_test = train_test_split(df, y, test_size=test_sep)
            self.x_train , self.x_dev , self.y_train, self.y_dev = train_test_split(
                                                                                    self.x_train, self.y_train, 
                                                                                    test_size= d_sep/(1-t_sep))
            
            
            
        else:
            self.t_sep = int(t_sep*len(df))
            self.d_sep = int((t_sep+d_sep)*len(df))

            self.x_train = df.iloc[0:self.t_sep,:]
            self.x_dev = df.iloc[self.t_sep:self.d_sep,:]
            self.x_test = df.iloc[self.d_sep:,:]
            
            self.y_train = y.iloc[0:self.t_sep]
            self.y_dev = y.iloc[self.t_sep:self.d_sep]
            self.y_test = y.iloc[self.d_sep:]       


In [10]:
df= df.set_index('bidtime').sort_index()

In [11]:
not_to_use = ['time', 'bidid', 'type', 'crid', 'deviceid', 'devlang', 'sspid',
                   'bundle', 'siteid', 'sitename', 'bid', 'impr',
                   'installprice', 'make', 'model', 'osv', 'isp', 'state', 'useroffset',
                   'height', 'uadevicetype', 'uabatterycapacity', 'uadeviceram',
                   'uareleaseyear', 'ppi', 'displaymanagerver','month', 'Impression_Price', 
                   'bundleCategory', 'bundleAvgUserRating','isLandscape',
                   'date', 'time_diff', 'local time','d_age_normalized',
                   'log2ofstorage', 'device_age','Local_day_of_week','install','normal_ssps']

cols_to_use = [x for x in df.columns if x not in not_to_use]

In [12]:
dummy_cals = ['ssps','bundletotalfc']
df_x = pd.get_dummies(df[cols_to_use].sort_index(),columns=dummy_cals)
df_y = df.install

In [13]:
df_x.isna().sum()

is_landscape             0
rewarded                 0
normal_rate          46037
cat_is_casino            0
ssps_1.0                 0
ssps_2.0                 0
ssps_3.0                 0
ssps_4.0                 0
ssps_5.0                 0
ssps_6.0                 0
ssps_7.0                 0
ssps_8.0                 0
ssps_9.0                 0
ssps_10.0                0
ssps_11.0                0
ssps_12.0                0
ssps_13.0                0
ssps_14.0                0
ssps_15.0                0
ssps_16.0                0
ssps_17.0                0
ssps_18.0                0
bundletotalfc_1.0        0
bundletotalfc_2.0        0
bundletotalfc_3.0        0
bundletotalfc_4.0        0
bundletotalfc_5.0        0
bundletotalfc_6.0        0
bundletotalfc_7.0        0
bundletotalfc_8.0        0
dtype: int64

### mean imputation for normal_rate - completely at random, we don't have all of the ratings for all of the apps in the store at a certain point


In [14]:
df_x.normal_rate = df_x.normal_rate.fillna(df.normal_rate.mean())

### Create a Data set class

In [ ]:
d_set = dataset(df_x,df_y,ts_orient=1)

## Check Data Imbalance - as a factor of time Since Data is organized by bid time

In [197]:
d_set.y_train.sum()/len(d_set.y_train)

0.004880211867832495

In [198]:
d_set.y_dev.sum()/len(d_set.y_dev)

0.007321451286907606

In [199]:
d_set.y_test.sum()/len(d_set.y_test)

0.005818493622624753

### Logistic Regression prediction

In [227]:
d_set.clf_log = LogisticRegression(class_weight='balanced',solver = 'newton-cg',C=.0015)
d_set.clf_log.fit(d_set.x_train,d_set.y_train)

LogisticRegression(C=0.0015, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='newton-cg', tol=0.0001, verbose=0, warm_start=False)

In [228]:
d_set.y_dev_pred = d_set.clf_log.predict(d_set.x_dev)

In [229]:
Confusion_Matrix = pd.crosstab(d_set.y_dev, d_set.y_dev_pred)
Confusion_Matrix

col_0,0.0,1.0
install,,
0.0,43060,27173
1.0,144,374


In [230]:
d_set.y_pred = d_set.clf_log.predict(d_set.x_test)
Confusion_Matrix2 = pd.crosstab(d_set.y_test, d_set.y_pred)
Confusion_Matrix2

col_0,0.0,1.0
install,,
0.0,43018,21911
1.0,121,259


In [231]:
roc_auc_score(d_set.y_test, d_set.y_pred)

0.6720590142593001

In [232]:
pd.DataFrame(list(zip(d_set.x_train.columns,d_set.clf_log.coef_[0])),
             columns=['feature','importance']).sort_values(by='importance',ascending = False)

,feature,importance
22,bundletotalfc_1.0,1.103366
3,cat_is_casino,0.991281
23,bundletotalfc_2.0,0.491416
1,rewarded,0.361005
4,ssps_1.0,0.355056
0,is_landscape,0.228060
7,ssps_4.0,0.178971
24,bundletotalfc_3.0,0.164204
18,ssps_15.0,0.128799
8,ssps_5.0,0.128174


### Light GBM

In [191]:
d_set.clf_lgbm = lgbm.LGBMClassifier(learning_rate=0.05,n_estimators=35,max_depth=3,class_weight='balanced',num_leaves=7,random_state=42)
d_set.clf_lgbm.fit(d_set.x_train,d_set.y_train)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
        colsample_bytree=1.0, importance_type='split', learning_rate=0.05,
        max_depth=3, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=35, n_jobs=-1, num_leaves=7,
        objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

dev-prediction

In [192]:
d_set.y_dev_pred = d_set.clf_lgbm.predict(d_set.x_dev)
Confusion_Matrix = pd.crosstab(d_set.y_dev, d_set.y_dev_pred)
Confusion_Matrix

col_0,0.0,1.0
install,,
0.0,44058,26175
1.0,146,372


In [193]:
roc_auc_score(d_set.y_dev, d_set.y_dev_pred)

0.6727293327609419

In [194]:
d_set.y_pred = d_set.clf_lgbm.predict(d_set.x_test)
Confusion_Matrix2 = pd.crosstab(d_set.y_test, d_set.y_pred)
Confusion_Matrix2

col_0,0.0,1.0
install,,
0.0,44413,20516
1.0,128,252


In [195]:
roc_auc_score(d_set.y_test, d_set.y_pred)

0.6735909912933236

In [196]:
pd.DataFrame(list(zip(d_set.x_train.columns,d_set.clf_lgbm.feature_importances_)),
             columns=['feature','importance']).sort_values(by='importance',ascending = False)

,feature,importance
3,cat_is_casino,53
22,bundletotalfc_1.0,35
2,normal_rate,32
1,rewarded,26
23,bundletotalfc_2.0,22
24,bundletotalfc_3.0,8
15,ssps_12.0,6
14,ssps_11.0,5
13,ssps_10.0,4
28,bundletotalfc_7.0,4


### Random Forest

In [184]:
d_set.clf_RF = RandomForestClassifier(class_weight='balanced',n_estimators=30,max_depth=5,min_samples_leaf=2,random_state=42)
d_set.clf_RF.fit(d_set.x_train,d_set.y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=None, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [185]:
d_set.y_dev_pred = d_set.clf_RF.predict(d_set.x_dev)
Confusion_Matrix = pd.crosstab(d_set.y_dev, d_set.y_dev_pred)
Confusion_Matrix

col_0,0.0,1.0
install,,
0.0,48755,21478
1.0,196,322


In [187]:
d_set.y_pred = d_set.clf_RF.predict(d_set.x_test)
Confusion_Matrix2 = pd.crosstab(d_set.y_test, d_set.y_pred)
Confusion_Matrix2

col_0,0.0,1.0
install,,
0.0,48956,15973
1.0,150,230


In [188]:
roc_auc_score(d_set.y_test, d_set.y_pred)

0.6796279904121992

In [189]:
pd.DataFrame(list(zip(d_set.x_train.columns,d_set.clf_RF.feature_importances_)),
             columns=['feature','importance']).sort_values(by='importance',ascending = False)

,feature,importance
22,bundletotalfc_1.0,0.395483
3,cat_is_casino,0.143908
27,bundletotalfc_6.0,0.079423
29,bundletotalfc_8.0,0.059359
28,bundletotalfc_7.0,0.059185
2,normal_rate,0.053148
1,rewarded,0.050079
26,bundletotalfc_5.0,0.032308
4,ssps_1.0,0.026782
23,bundletotalfc_2.0,0.018934


### Decision Trees

In [178]:
d_set.clf_DT = tree.DecisionTreeClassifier(max_leaf_nodes=3,class_weight='balanced',random_state=42)
d_set.clf_DT.fit(d_set.x_train,d_set.y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=3,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [179]:
d_set.y_dev_pred = d_set.clf_DT.predict(d_set.x_dev)
Confusion_Matrix = pd.crosstab(d_set.y_dev, d_set.y_dev_pred)
Confusion_Matrix

col_0,0.0,1.0
install,,
0.0,50177,20056
1.0,204,314


In [180]:
d_set.y_pred = d_set.clf_DT.predict(d_set.x_test)
Confusion_Matrix = pd.crosstab(d_set.y_test, d_set.y_pred)
Confusion_Matrix

col_0,0.0,1.0
install,,
0.0,49924,15005
1.0,157,223


In [181]:
roc_auc_score(d_set.y_test, d_set.y_pred)

0.6778717603276777

In [182]:
pd.DataFrame(list(zip(d_set.x_train.columns,d_set.clf_DT.feature_importances_)),
             columns=['feature','importance']).sort_values(by='importance',ascending = False)

,feature,importance
22,bundletotalfc_1.0,0.838413
3,cat_is_casino,0.161587
0,is_landscape,0.000000
16,ssps_13.0,0.000000
28,bundletotalfc_7.0,0.000000
27,bundletotalfc_6.0,0.000000
26,bundletotalfc_5.0,0.000000
25,bundletotalfc_4.0,0.000000
24,bundletotalfc_3.0,0.000000
23,bundletotalfc_2.0,0.000000
